# Анализ теста компьютерной кампеметрии

In [3]:
import pandas as pd

Фильтрация данных


In [5]:
file_path = "D:/AI/Computerized-campimetry-test-analysis/data.xlsx"
data = pd.read_excel(file_path)
necessary = data[data["id_test_attempt"]==0]
test_data = necessary.rename(columns=lambda x: x.replace('T258', 'CM')) #замена названий столбцов

In [6]:
data.head()

,Unnamed: 0,T258_dH+_red_mean,T258_dH+_red_std,T258_dH+_red_min,T258_dH+_red_25%,T258_dH+_red_50%,T258_dH+_red_75%,T258_dH+_red_max,T258_dH+_yellow_green_mean,T258_dH+_yellow_green_std,...,T278_SMR_diff_25%,T278_SMR_diff_50%,T278_SMR_diff_75%,T278_SMR_diff_max,T278_SMR_corr,T278_SMR_tau,id_test_attempt,id_sample,age,sex
0,0,1975-02-01 00:00:00,2025-05-01 00:00:00,1.0,1975-01-01 00:00:00,3.0,4.0,4.0,1.0,0.816496580927726,...,-0.111650,-0.003722,0.110599,0.298893,-0.348606,-0.231483,0,0,20.01095890410959,1
1,1,2025-02-01 00:00:00,0.9574271077563381,1.0,1975-01-01 00:00:00,2025-05-02 00:00:00,3.0,3.0,0.75,0.5,...,-0.051903,0.048618,0.063985,0.219723,0.102884,0.066421,0,1,20.013698630136986,0
2,2,1975-01-01 00:00:00,0.9574271077563381,1.0,1.0,2025-05-01 00:00:00,2025-02-01 00:00:00,3.0,0.5,0.5773502691896257,...,-0.064603,-0.051664,0.073810,0.274372,-0.202019,-0.016667,0,2,20.112328767123287,0
3,3,2.0,0.816496580927726,1.0,1975-01-01 00:00:00,2.0,2025-02-01 00:00:00,3.0,0.75,0.5,...,-0.055315,-0.027076,0.070588,0.181467,0.019172,-0.049460,0,3,20.15890410958904,0
4,4,2.0,0.816496580927726,1.0,1975-01-01 00:00:00,2.0,2025-02-01 00:00:00,3.0,0.25,0.5,...,-0.129545,0.003198,0.072435,0.324830,-0.149060,-0.191063,1,4,20.15890410958904,0


In [17]:
test_data = test_data[["CM_dH+_yellow_green_50%", "CM_dH+_blue_50%","CM_dH+_purple_50%","CM_dH+_red_50%"]]
test_data

,CM_dH+_yellow_green_50%,CM_dH+_blue_50%,CM_dH+_purple_50%,CM_dH+_red_50%
0,1.0,4457.5,4610.0,3.0
1,1.0,4519.0,9925.5,2025-05-02 00:00:00
2,0.5,9886.5,6593.0,2025-05-01 00:00:00
3,1.0,3710.0,4141.0,2.0
6,0.0,8748.5,7961.5,3.0
...,...,...,...,...
2988,10.0,16521.0,2616.5,2025-05-07 00:00:00
2989,2025-05-05 00:00:00,11435.0,9460.5,2025-05-07 00:00:00
2990,2025-05-06 00:00:00,29358.5,16345.5,10.0
2991,2025-05-03 00:00:00,15058.5,9581.5,13.0
